In [3]:
import pandas as pd
import sqlite3

# Step 1: Connect to the SQLite database
conn = sqlite3.connect('../interface/database.db')

# Step 2: Retrieve table names from the database
query = "SELECT * FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)

# Step 3 & 4: Query each table and save it as a CSV file
for table_name in tables['name'].tolist():
    table_query = f"SELECT * FROM {table_name}"
    table_df = pd.read_sql_query(table_query, conn)
    print(table_df.head())
    table_df.to_csv(f"{table_name}.csv", index=False)  # index=False to exclude the index from the CSV

# Close the connection
conn.close()


   ID                     name
0   1  Teaching and explaining
1   2             Social Media
2   3             Econometrics
3   4   Fantasy story building
4   5     Freelancer Marketing
   ID                                             prompt  isSelected  UCID  \
0   1  Advise on creating interactive and engaging ac...           1     1   
1   2  Based on the classroom personalities and learn...           1     1   
2   3  Recommend the use of analogies and examples th...           1     1   
3   4  Provide guidance on how to facilitate a classr...           1     1   
4   5  Suggest how to use technology and multimedia t...           1     1   

                                   keyFeatures  
0  Develop Interactive and Engaging Activities  
1       Identify Effective Teaching Strategies  
2                   Use Analogies and Examples  
3             Facilitate Inclusive Discussions  
4  Incorporate Technology and Multimedia Tools  
   ID  testID                                      

In [4]:
import pandas as pd

# Load the datasets
interaction_df = pd.read_csv('./interaction.csv')
system_prompt_df = pd.read_csv('./systemPrompt.csv')
test_df = pd.read_csv('./test.csv')
use_case_df = pd.read_csv('./useCase.csv')

# Display the first few rows of each dataset to understand their structure
(interaction_df.head(), system_prompt_df.head(), test_df.head(), use_case_df.head())


(   ID  testID                                              human  \
 0   1       1  Help me to sell a guide containing also video-...   
 1   2       1  Your last answer about the promotional campaig...   
 
                                                   ai  pertinence  \
 0   To develop the perfect promotional campaign f...           4   
 1   Day 16: Social Media Giveaway\nHost a giveawa...           4   
 
    chosenPromptID excludedPromptIDs  
 0              19             20,21  
 1              19             20,21  ,
    ID                                             prompt  isSelected  UCID  \
 0   1  Advise on creating interactive and engaging ac...           1     1   
 1   2  Based on the classroom personalities and learn...           1     1   
 2   3  Recommend the use of analogies and examples th...           1     1   
 3   4  Provide guidance on how to facilitate a classr...           1     1   
 4   5  Suggest how to use technology and multimedia t...           1

In [5]:
# Explore how the 'testID' in 'interaction' corresponds to entries in 'test'
interaction_test_merged = pd.merge(interaction_df, test_df, left_on="testID", right_on="ID", suffixes=('_interaction', '_test'))

# Explore how the 'UCID' in 'systemPrompt' matches entries in 'useCase'
system_prompt_use_case_merged = pd.merge(system_prompt_df, use_case_df, left_on="UCID", right_on="ID", suffixes=('_prompt', '_useCase'))

# Show the result of merging to confirm correctness of data connections
(interaction_test_merged.head(), system_prompt_use_case_merged.head())


(   ID_interaction  testID                                              human  \
 0               1       1  Help me to sell a guide containing also video-...   
 1               2       1  Your last answer about the promotional campaig...   
 
                                                   ai  pertinence  \
 0   To develop the perfect promotional campaign f...           4   
 1   Day 16: Social Media Giveaway\nHost a giveawa...           4   
 
    chosenPromptID excludedPromptIDs  ID_test  loadingTime     model  \
 0              19             20,21        1          0.0  OPENCHAT   
 1              19             20,21        1          0.0  OPENCHAT   
 
    withTranslation                 timestamp  
 0                0  2024-04-26T14:38:43.799Z  
 1                0  2024-04-26T14:38:43.799Z  ,
    ID_prompt                                             prompt  isSelected  \
 0          1  Advise on creating interactive and engaging ac...           1   
 1          2  Based on

In [8]:
# Attempt to re-run the analysis on chosen prompts effectiveness
interaction_prompt_merged = pd.merge(interaction_df, system_prompt_df, left_on="chosenPromptID", right_on="ID", suffixes=('_interaction', '_prompt'))

# Aggregate data to analyze effectiveness of chosen prompts per use case
chosen_prompt_analysis = interaction_prompt_merged.groupby(['UCID', 'chosenPromptID', 'prompt', 'keyFeatures']) \
                                                  .agg(
                                                      average_pertinence=('pertinence', 'mean'),
                                                      count=('ID_interaction', 'size')
                                                  ).reset_index()

# Examine the best performing prompts per use case based on average pertinence and frequency
best_prompts_per_use_case = chosen_prompt_analysis.sort_values(by=['UCID', 'average_pertinence', 'count'], ascending=[True, False, False])

# Show the analysis results
best_prompts_per_use_case.head(10)


,UCID,chosenPromptID,prompt,keyFeatures,average_pertinence,count
0,5,19,You are the best marketing strategist. You alw...,Step-by-step guide,4.0,2


In [9]:
# Expand the 'excludedPromptIDs' into a list and explode it for analysis
interaction_df['excludedPromptIDs_list'] = interaction_df['excludedPromptIDs'].str.split(',')

# Explode the DataFrame on the excludedPromptIDs_list to analyze each excluded prompt individually
excluded_prompts_expanded = interaction_df.explode('excludedPromptIDs_list')

# Convert the exploded 'excludedPromptIDs_list' to numeric for merging
excluded_prompts_expanded['excludedPromptIDs_list'] = pd.to_numeric(excluded_prompts_expanded['excludedPromptIDs_list'])

# Merge with system prompt data to include prompt text and use case info
excluded_prompt_analysis = pd.merge(excluded_prompts_expanded, system_prompt_df, left_on='excludedPromptIDs_list', right_on='ID', suffixes=('_interaction', '_prompt'))

# Aggregate data to analyze frequently excluded prompts per use case
frequent_excluded_prompts = excluded_prompt_analysis.groupby(['UCID', 'excludedPromptIDs_list', 'prompt', 'keyFeatures']) \
                                                    .agg(count=('ID_interaction', 'size')) \
                                                    .reset_index().sort_values(by='count', ascending=False)

# Show the analysis results for excluded prompts
frequent_excluded_prompts.head(10)


,UCID,excludedPromptIDs_list,prompt,keyFeatures,count
0,5,20,Plan a promotional campaign focusing on budget...,"Budgeting, Metrics Evaluation Prompt",2
1,5,21,You're tasked with creating an innovative prom...,Creative Concept Development Prompt,2
